In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import pymongo
from pymongo import MongoClient
import seaborn as sns
conn = MongoClient() 

In [3]:
# HYPOTHESIS 1: Young people use more money related to their incomes than old people WRONG
# HYPOTHESIS 2: The more work experience you have the more money you make WRONG
# HYPOTHESIS 3: People with small income take more loans than people with big incomes WRONG
# HYPOTHESIS 4: Online banking usage is relatively bigger in young people than in old people WRONG

In [14]:
# Data Description:								
								
# ID	        Customer ID							
# Age	        Customer's age in completed years							
# Experience	years of professional experience							
# Income	    Annual income of the customer ($000)							
# ZIPCode	    Home Address ZIP code.							
# Family	    Family size of the customer							
# CCAvg	Avg.    spending on credit cards per month ($000)							
# Education	    Education Level. 1: Undergrad; 2: Graduate; 3: Advanced/Professional							
# Mortgage	    Value of house mortgage if any. ($000)							
# Personal Loan	Did this customer accept the personal loan offered in the last campaign?							
# Securities    Account	Does the customer have a securities account with the bank?							
# CD Account	Does the customer have a certificate of deposit (CD) account with the bank?							
# Online	    Does the customer use internet banking facilities?							
# CreditCard	Does the customer use a credit card issued by UniversalBank?		

					
db = conn.bank_loans


df = pd.DataFrame(db.bank_loans_hadoop.find({}))
df

,_id,ZIPCode,ID,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,649ada5871bff09848769564,9307,385,51,25,21,4,0.600000,3,0,0,0,0,1,1
1,649ada5871bff09848769565,90005,653,34,9,92,1,2.800000,1,0,0,0,0,1,0
2,649ada5871bff09848769566,90005,1470,59,35,59,4,1.200000,2,0,0,0,0,1,0
3,649ada5871bff09848769567,90005,1682,32,8,141,2,4.333333,1,0,0,0,0,0,0
4,649ada5871bff09848769568,90005,1620,45,21,29,1,0.300000,3,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,649ada5b71bff0984876a8e7,96651,1654,26,1,24,2,0.900000,3,123,0,0,0,0,1
4996,649ada5b71bff0984876a8e8,96651,631,32,7,35,3,1.300000,1,108,0,0,0,0,1
4997,649ada5b71bff0984876a8e9,96651,673,51,27,23,1,0.200000,1,0,0,0,0,1,0
4998,649ada5b71bff0984876a8ea,96651,2732,29,5,28,1,0.200000,3,0,0,0,0,1,0


In [16]:
# HYPOTHESIS 1: Average credit card usage for young and old people, compared to average income.


# df[(df.Mortgage < 200) & (df.Income > 30)]

# Split the df to two dataframes containing young and old customers.
young_df = df[(df.Age < 44)]
old_df = df[(df.Age >= 44)]

# Get income averages
young_income_avg = np.average(young_df.Income)
old_income_avg = np.average(old_df.Income)

# GEt credit card usage averages
young_ccuse_avg = np.average(young_df.CCAvg)
old_ccuse_avg = np.average(old_df.CCAvg)

# Print comparison
print("Income avg: ", young_income_avg, old_income_avg)
print("CC use avg: ", young_ccuse_avg, old_ccuse_avg)


Income avg:  75.93571114046675 71.9754488823745
CC use avg:  2.016563921913988 1.8724624404543788


In [18]:
# HYPOTHESIS 2: Compare if experience affects income

# Get dataframe containing dataframe with high experience customers
bigexp_df = df[df.Experience >= 10]
# Get dataframe containing dataframe with low experience customers
smallexp_df = df[df.Experience < 10]

print("Sizes: ", bigexp_df.shape, smallexp_df.shape)

# Get average incomes for both
bigexp_avg_income = np.average(bigexp_df.Income)
smallexp_avg_income = np.average(smallexp_df.Income)

# Get median income for both
bigexp_median_income = np.median(bigexp_df.Income)
smallexp_median_income = np.median(smallexp_df.Income)

# Print comparison
print("Avg incmes: ", bigexp_avg_income, smallexp_avg_income)
print("Avg incmes: ", bigexp_median_income, smallexp_median_income)

Sizes:  (3829, 15) (1171, 15)
Avg incmes:  73.12744841995298 75.88898377455166
Avg incmes:  63.0 68.0


In [19]:
from sklearn.preprocessing import scale

mydata_std = pd.DataFrame(df, index=df.index, columns=df.columns)

mydata_std.describe()

corr = mydata_std.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

# sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
#             square=True)

/tmp/ipykernel_3592/1715726638.py:8: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  corr.style.background_gradient(cmap='coolwarm').set_precision(2)


,ZIPCode,ID,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
ZIPCode,1.00,0.01,-0.03,-0.03,-0.02,0.01,-0.00,-0.02,0.01,0.00,0.00,0.02,0.02,0.01
ID,0.01,1.00,-0.01,-0.01,-0.02,-0.02,-0.02,0.02,-0.01,-0.02,-0.02,-0.01,-0.00,0.02
Age,-0.03,-0.01,1.00,0.99,-0.06,-0.05,-0.05,0.04,-0.01,-0.01,-0.00,0.01,0.01,0.01
Experience,-0.03,-0.01,0.99,1.00,-0.05,-0.05,-0.05,0.01,-0.01,-0.01,-0.00,0.01,0.01,0.01
Income,-0.02,-0.02,-0.06,-0.05,1.00,-0.16,0.65,-0.19,0.21,0.50,-0.00,0.17,0.01,-0.00
Family,0.01,-0.02,-0.05,-0.05,-0.16,1.00,-0.11,0.06,-0.02,0.06,0.02,0.01,0.01,0.01
CCAvg,-0.00,-0.02,-0.05,-0.05,0.65,-0.11,1.00,-0.14,0.11,0.37,0.02,0.14,-0.00,-0.01
Education,-0.02,0.02,0.04,0.01,-0.19,0.06,-0.14,1.00,-0.03,0.14,-0.01,0.01,-0.02,-0.01
Mortgage,0.01,-0.01,-0.01,-0.01,0.21,-0.02,0.11,-0.03,1.00,0.14,-0.01,0.09,-0.01,-0.01
Personal Loan,0.00,-0.02,-0.01,-0.01,0.50,0.06,0.37,0.14,0.14,1.00,0.02,0.32,0.01,0.00


In [20]:
# HYPOTHESIS 3: People who took personal loan

# Take subset of the dataframe to get only the people who took personal loan
ploan_df = df[df["Personal Loan"] > 0]
# Count all the personal loan takers.
ploan_all_amount = ploan_df.shape[0]

# Define limit for "rich". This dataset is suspected to be from USA, and various 
# sources stated that average salary in the USA is somewhere between 50K-70K. So we set the limit to 80K.
rich_limit = 80

# Count all the personal loan takers who had high income.
rich_ploan_df = ploan_df[ploan_df.Income > rich_limit]
ploan_rich_amount = rich_ploan_df.shape[0]

# Print comparison
print(f"Amount of personal loans for >{rich_limit}K/year: {ploan_rich_amount}/{ploan_all_amount}")


Amount of personal loans for >80K/year: 473/480


In [21]:
# HYPOTHESIS 4 Online banking usage is relatively bigger in young people than in old people

db = conn.bank_loans
collection = db. bank_loans
young_total = collection.count_documents({"Age": {"$lt": 44}})
old_total = collection.count_documents({"Age": {"$gte": 44}})

print("Total of under 44 years old in the dataset:", young_total)
print("Total of at least 44 years old in the dataset:", old_total)

young_online_usage = collection.count_documents({"$and": [{"Age": {"$lt": 44}}, {"Online": 1}]})
old_online_usage = collection.count_documents({"$and": [{"Age": {"$gte": 44}}, {"Online": 1}]})

print("Online banking usage in under 44 years olds:", young_online_usage)
print("Online banking usage in at least 44 years olds:", old_online_usage)

young_online_usage_relation = round((young_online_usage / young_total) * 100, 1)
old_online_usage_relation = round((old_online_usage / old_total) * 100, 1)
print(f"Percentage of under 44 year olds to use online services {young_online_usage_relation}%")
print(f"Percentage of at least 44 year olds to use online services {old_online_usage_relation}%")
print(f"Hypothesis proved WRONG since the online services usage is slightly bigger {old_online_usage_relation} among the older people than among the younger people {young_online_usage_relation}")

Total of under 44 years old in the dataset: 2271
Total of at least 44 years old in the dataset: 2729
Online banking usage in under 44 years olds: 1339
Online banking usage in at least 44 years olds: 1645
Percentage of under 44 year olds to use online services 59.0%
Percentage of at least 44 year olds to use online services 60.3%
Hypothesis proved WRONG since the online services usage is slightly bigger 60.3 among the older people than among the younger people 59.0
